# KI Implementierung (othello_ai.ipynb)
\label{sec:aiimpl}
\ifx false

In [ ]:
%%HTML
<style>
.container { width:100% }
</style>

\fi
Dieses Kapitel beschreibt die Implementierung der \ac{KI}. Es werden mehrere unterschiedliche Strategien implementiert, um einen Vergleich von diesen zu ermöglichen.

## Importieren der externen Abhängigkeiten
Zur Initialisierung der Parameter `alpha` und `beta` in der mit Alpha-Beta Pruning optimierten Minimax Strategie werden die Konstanten `math.inf` und `-math.inf` benötigt. Sie stehen jeweils für den maximalen und den minimalen Wert, den eine Fließkommazahl annehmen kann. Die Konstanten werden von der Python Standardbibliothek in dem Modul `math` bereitgestellt

Das Modul `random` wird im Rahmen dieser Implementierung für mehrere Zwecke genutzt.
Zum einen zur Implementierung der `random_ai`, einer Strategie, welche immer einen zufälligen Zug wählt und zum anderen, um die auf Minimax basierenden Strategien nicht-deterministisch zu machen.

In [ ]:
import math
import random

Die Implementierung der \ac{KI} baut auf der Implementierung der Spielelogik von Othello auf. Das entsprechende Notebook muss also hier ausgeführt werden.

In [ ]:
%run othello_game.ipynb

Die aktuellen Nutzen-Werte der beiden Spieler werden in der globalen Variable utilities gespeichert, sodass diese in der GUI angezeigt werden können. Diese Werte werden in den entsprechenden Funktionen der Strategien aktualisiert.

In [ ]:
utilities = {WHITE: '-', BLACK: '-'}

## Heuristiken

Zum Abschätzen der Nützlichkeit eines Spielzustands wird eine Heuristik benötigt. Im Folgenden sind einige solcher Heuristiken implementiert. Da Weiß der maximierende Spieler und Schwarz der minimierende Spieler ist, repräsentiert ein höherer Wert der Heuristik einen für Weiß vorteilhaften Zug, während ein niedriger Wert einen Vorteil für Schwarz repräsentiert. Die Werte aller Heuristiken liegen zwischen $-1$ und $1$, wobei diese Randwerte einen garantierten Sieg für den jeweiligen Spieler darstellen. Der Wert $0$ steht für einen für beide Spieler gleich guten Spielzustand.

Die Funktion `disc_count_heuristic` implementiert die in \ref{sec:disccount} beschriebene Heuristik. Dafür wird die Differenz der Anzahlen an Steinen beider Spieler auf dem Spielfeld bestimmt. Diese wird zur Normalisierung durch die maximale Steinzahl $64$ geteilt.

In [ ]:
def disc_count_heuristic(state):
    return (count_disks(state, WHITE) - count_disks(state, BLACK)) / 64

`mobility_heuristic` berechnet wie in \ref{sec:theorycurrentmobility} beschrieben die aktuelle Mobilität. Auch dieser Wert wird durch Division durch die Anzahl an Feldern normalisiert, um die Grenzen von $-1$ und $1$ einzuhalten. Zu beachten ist hier, dass auch die Anzahl möglicher Züge für einen Spieler bestimmt wird, der im Spielzustand gar nicht am Zug ist. Dies wirkt zunächst sematisch nicht sinvoll, hat sich jedoch, wie in \ref{sec:currentmobility} gezeigt, im Vergleich gegenüber möglicher Alternativen, wie der Verwendung einer durchschnittlichen Mobilität, als effektiver erwiesen.

In [ ]:
def mobility_heuristic(state):
    if state.turn == WHITE:
        return (len(state.possible_moves) -
                len(get_possible_moves(state, BLACK))) / 64
    else:
        return (len(get_possible_moves(state, WHITE)) -
                len(state.possible_moves)) / 64

Nicht nur die aktuelle, sondern auch die potenzielle Mobilität, welche in \ref{sec:potmobility} beschrieben wird, kann vor allem in frühen Phasen des Spiels wichtig für die Bewertung einer Position sein. Die Funktion `pot_mob_heuristic` berechnet für einen Zustand `state` die Differenz der potenziellen Mobilität beider Spieler. Die potenzielle Mobilität eines Spielers ist hierbei gegeben durch die Summe aller freier Felder um gegnerische Spielsteine, da Michael Buro dieses Merkmal in seiner Dissertation als beste Metrik für die potenzielle Mobilität ausgemacht hat \cite[S. 9]{evaluationfunctions}. Das Ergebnis wird durch $3.5$ geteilt, da es in zufälligen Spielen im Durchschnitt $3.5$ mal so viele potenzielle Züge wie tatsächliche Züge gibt.

In [ ]:
def pot_mob_heuristic(state):
    board = list(state.board)
    fields = 0
    for (x,y) in state.frontier:
        for dx, dy in directions:
            xi = x + dx
            yi = y + dy
            if 0 <= xi < 8 and 0 <= yi < 8 and board[xi][yi] != 0:
                fields -= board[xi][yi]
    fields /= 3.5 
    return fields / 64

Die Funktion `combined_mobility_heuristic` kombiniert die aktuelle und potenzielle Mobilität, wobei zu Beginn des Spiels die potenzielle Mobilität stärker gewichtet wird und gegen Ende des Spiels die aktuelle Mobilität. Michael Buro beschreibt in seiner Dissertation, dass die potenzielle Mobilität, bis 36 Spielsteine auf dem Feld liegen, wichtiger für die Bewertung ist, als die aktuelle Mobilität \cite[S. 9]{evaluationfunctions}. Eigene Tests, welche in Kapitel \ref{sec:combinedmobility} beschrieben sind, ergeben, dass eine lineare Kombination der beiden Merkmale zu einem guten Ergebnis führt.

In [ ]:
def combined_mobility_heuristic(state):
    act = mobility_heuristic(state)
    pot = pot_mob_heuristic(state)
    return (1 - state.num_pieces / 50) * pot + (state.num_pieces / 50) *  act

Beim Spielen von Othello fällt auf, dass es bestimmte Felder gibt, deren Belegung von Vorteil ist, sowie einige, deren Belegung eher nachteilhaft ist. Diese Eigenschaft macht sich die Cowthello-Heuristik \cite{cowthello} zunutze, welche in \ref{sec:cowthello} beschrieben ist. Sie weist jedem Feld einen Wert zu, der angibt, wie Vorteilhaft der Besitz dieses Feldes ist, bzw. wie Nachteilhaft die Belegung des Feldes durch den Gegner ist. Diese Gewichte werden mit der aktuellen Belegung des Spielfelds multipliziert und die Ergebnisse anschließend aufsummiert. Der resultierende Wert schätzt dann den Nutzen der aktuellen Position ein. Auch bei dieser Heuristik findet eine Normalisierung statt.

Da die Ecken des Spielfeldes, sobald sie einmal belegt sind, nicht mehr vom Gegner erobert werden können, spielen diese eine besondere Rolle. Ist eine Ecke unbelegt, so sind die umliegenden Spielfelder anders zu bewerten, als bei einer belegten Ecke. Dies wird von der Cowthello Heuristik berücksichtigt.

Aufgrund der Symmetrie des Othello Spielfeldes, ist es für die Cowthello-Heuristik nicht nötig, für jedes Feld einzeln dessen Gewicht anzugeben. Stattdessen werden ausschließlich die Gewichte für ein Viertel des Spielfelds angegeben und dieses anschließend gespiegelt. Die Funktion `gen_cowthello_matrix` generiert dann die Gewichte-Matrix für das gesamte Feld und führt auch die Normalisierung durch. Dabei werden die Gewichte aus dem Online-Othello Programm Cowthello \cite{cowthello} verwendet. Cowthello ist unter der URL <https://www.aurochs.org/games/cowthello/> verfügbar. Die Funktion `gen_cowthello_matrix` erzeugt hierbei die bei unbelegten Ecken verwendeten Gewichte. Sind Ecken belegt, werden die Gewichte später entsprechend angepasst.

In [ ]:
def gen_cowthello_matrix():
    quarter = np.array([
        [100, -25, 25, 10],
        [-25, -50,  1,  1],
        [ 25,   1, 50,  5],
        [ 10,   1,  5,  1]
    ])
    top = np.hstack((quarter, np.flip(quarter, axis=1)))
    bottom = np.flip(top, axis=0)
    cowthello_matrix = np.vstack((top, bottom))
    return cowthello_matrix;

Die Funktion `modify_corner_weights` übernimmt die Modifikation der Gewichte-Matrix, für eine belegte Ecke. Die Ecke wird hierbei durch deren Koordinaten `row` und `column`, sowie die Richtung zum Mittelfeld des Spielfels `rdir`, `cdir` spezifiziert. Die als Parameter übergebene Gewichte-Matrix `weights` wird hierbei verändert.

In [ ]:
def modify_corner_weights(weights, row, col, rdir, cdir):
    weights[row+rdir   , col       ] = 10
    weights[row+2*rdir , col       ] = 10
    weights[row        , col+cdir  ] = 10
    weights[row        , col+2*cdir] = 10
    weights[row+rdir   , col+cdir  ] = 1
    weights[row+2*rdir , col+2*cdir] = 5

Um den Wert der Cowthello Heuristik zwischen 0 und 1 einzugrenzen, wird der maximal mögliche Wert der Heuristik berechnet und in der Variable `norm_factor` gepeichert.

In [ ]:
norm_factor = np.sum(np.abs(gen_cowthello_matrix()))

Die Funktion `cowthello_heuristic` bestimmt aus einem Spielzustand und der aufgestellten Gewichtematrix `cowthello_weights` die gewichtete Summe, welche als Heuristik genutzt wird. Für jede belegte Ecke wird die Gewichte-Matrix mittels `modify_corner_weights` entsprechend modifiziert.

In [ ]:
def cowthello_heuristic(state):
    weights = gen_cowthello_matrix()
    if state.board[0, 0] != NONE:
        modify_corner_weights(weights, 0, 0, 1, 1)
    if state.board[7, 0] != NONE:
        modify_corner_weights(weights, 7, 0,-1, 1)
    if state.board[0, 7] != NONE:
        modify_corner_weights(weights, 0, 7, 1,-1)
    if state.board[7, 7] != NONE:
        modify_corner_weights(weights, 7, 7,-1,-1)
    heuristic = np.sum(np.multiply(state.board,weights))
    return heuristic / norm_factor

Die oben implementierten Heuristiken bewerten jeweils nur ein Merkmal der aktuellen Spielsitation. Durch eine Kombination mehrerer dieser Heuristiken können mehrere Merkmale gleichzeitig betrachtet werden. Die Gewichtung von Mobilität und Cowthello wird in Kapitel \ref{sec:mobcowweight} bestimmt.

In [ ]:
def combined_heuristic(state):
    mobility = combined_mobility_heuristic(state)
    cowthello = cowthello_heuristic(state)
    return 0.625 * mobility + 0.375 * cowthello

## Implementierung der Strategien

Im Folgenden werden die verschiedenen Strategien der \ac{KI} implementiert. Diese verwenden zum Teil die im vorherigen Kaptitel implementierten Heuristiken.
Die Strategien bestehen jeweils aus einer bewertenden Funktion, die den Nutzen einer Spielsituation bestimmt, sowie aus einer aufrufenden Funktion, welche mithilfe der bewertenden Funktion den bestmöglichen Zug ermittelt. Diese Komponenten können beliebig kombiniert werden. Im Folgenden werden zunächst die bewertenden Funktionen implementiert.

### Zufällige KI
Die zufällige \ac{KI} bewertet den Nutzen aller Züge gleich, gibt also immer den Wert $0$ zurück.  Da die Strategie-Funktionen völlig austauschbar sein sollen, müssen alle diese Funktionen dieselben Eingabeparameter haben. Im Fall der Funktion `random_ai` wird jedoch keiner der definierten Parameter benötigt. Der Zweck dieser \ac{KI} ist die Messung der Stärke der anderen \acp{KI}.

In [ ]:
def random_ai(state, depth, heuristic, alpha, beta):
    return 0

### Minimax KI
Die Minimax Strategie verwendet den unveränderten Minimax Algorithmus, wie er in \autoref{sec:minimax} beschrieben ist, zur Bestimmung der Nützlichkeit eines Zuges. Eingabeparameter sind hier der zu bewertende Spielzustand `state`, die gewünschte Suchtiefe `depth` sowie die zu verwendende Heuristik
`heuristic`. Die Parameter `alpha` und `beta` dienen, wie oben beschrieben, der Kompatibilität mit den folgenden Strategie-Funktionen und werden in der Funktion `minimax` nicht verwendet. Der Rückgabeparameter gibt die ermittelte Nützlichkeit des Spielzustands an.

In [ ]:
debug_mm_count = 0

def minimax(state, depth, heuristic, alpha, beta):
    global debug_mm_count
    if state.game_over:
        return get_utility(state)
    if depth == 0:
        debug_mm_count += 1
        return heuristic(state)

    if state.turn == WHITE:
        # maximizing
        utility = -math.inf
    else:
        # minimizing
        utility = math.inf

    for move in state.possible_moves:
        tmp_state = make_move(state, move)
        tmp_utility = minimax(tmp_state, depth - 1, heuristic, None, None)
        if state.turn == WHITE:
            # maximizing
            utility = max(utility, tmp_utility)
        else:
            # minimizing
            utility = min(utility, tmp_utility)
    return utility

### Alpha-Beta KI
Diese \ac{KI} verwendet den Minimax Algorithmus mit der Optimierung Alpha-Beta Pruning, welche in \autoref{sec:alphabeta} beschrieben ist, um die Nützlichkeit eines Spielzustands zu bestimmen.

Zum Merken der Ergebnisse vorheriger Ausführungen wird das Dictionary `transposition_table` verwendet. Dies ist gerade bei der Verwendung von Iterative Deepening für das Move Ordering vorteilhaft. Der Schlüssel des Dictionaries besteht aus dem Zustand des Spielbretts, dem Spieler, der an der Reihe ist, und der verwendeten Heuristik.

In [ ]:
transposition_table = {}

Die Funktion `alphabeta` implementiert den Minimax-Algorithmus mit Alpha-Beta-Pruning. Eingabeparameter der Funktion sind der zu bewertende Spielzustand `state`, die maximale Suchtiefe `depth`, die zu verwendende Heuristik `heuristic`, sowie die Werte `alpha` und `beta`, die, wie in \autoref{sec:alphabeta} beschrieben, jeweils den sicher erreichbaren Nutzen für den maximierenden und minimierenden Spieler angeben und für das Abschneiden von Zweigen verwendet werden.

In [ ]:
debug_ab_count = 0

def alphabeta(state, depth, heuristic, alpha, beta):
    global debug_ab_count
    if state.game_over:
        return get_utility(state)
    if depth == 0:
        key = (state.board.tobytes(), state.turn, heuristic)
        if key in transposition_table:
            return transposition_table[key][0]
        debug_ab_count += 1
        h = heuristic(state)
        transposition_table[key] = (h, 0)
        return h

    moves = state.possible_moves
    child_states = [make_move(state, move) for move in moves]
    ordered_moves = []
    for child_state in child_states:
        key = (child_state.board.tobytes(), child_state.turn, heuristic)
        cached = transposition_table.get(key)
        if cached == None:
            debug_ab_count += 1
            cached = (heuristic(child_state), 0)
            transposition_table[key] = cached
        ordered_moves.append((cached[0], child_state, cached[1]))
    ordered_moves.sort(reverse=(state.turn == WHITE))

    if state.turn == WHITE:
        # maximizing
        utility = -math.inf
    else:
        # minimizing
        utility = math.inf

    for (_, tmp_state, cached_depth) in ordered_moves:
        tmp_utility = alphabeta(tmp_state, depth-1, heuristic, alpha, beta)
        if depth - 1 > cached_depth:
            transposition_table[
                (tmp_state.board.tobytes(),
                 tmp_state.turn, heuristic)
            ] = (tmp_utility, depth -1)

        if state.turn == WHITE:
            # maximizing
            utility = max(utility, tmp_utility)
            alpha = max(alpha, utility)
        else:
            # minimizing
            utility = min(utility, tmp_utility)
            beta = min(beta, utility)
        if alpha >= beta:
            break  # alpha-beta pruning
    return utility

### ProbCut KI
An dieser Stelle beginnt die Implementierung der \ac{KI} mittels des Minimax Algorithmus, Alpha-Beta Pruning und ProbCut. Die im Folgenden definierte Konstante `PERCENTILE` entspricht hierbei dem Term $\Phi^{-1}(p)$ aus \autoref{sec:probcut}. Für ein $p$ von $93.3\%$ hat `PERCENTILE` den Wert $1.5$. `PROBCUT_DEEP_DEPTH` und `PROBCUT_SHALLOW_DEPTH` entsprechen den Variablen $d$ und $d'$ aus \autoref{sec:probcut} dieser Arbeit.

In [ ]:
PERCENTILE = 1.5
PROBCUT_DEEP_DEPTH = 4
PROBCUT_SHALLOW_DEPTH = 2

Die Implementierung der ProbCut Strategie gleicht in großen Teilen der Implementierung der Alpha-Beta Strategie. Jedoch wird  bei jedem Aufruf mit der Tiefe `PROBCUT_DEEP_DEPTH`, zunächst eine Suche mit der Tiefe `PROBCUT_SHALLOW_DEPTH` durchgeführt. Anhand der dabei ermittelten Nützlichkeit wird entsprechend der in \autoref{sec:probcut} beschriebenen Regeln entschieden, ob eine Tiefe Suche durchgeführt werden muss, oder einer der beiden Grenzwerte `alpha` oder `beta` zurückgegeben werden kann. Zur Abschätzung der für den Probcut Algorithmus benötigten Standardabweichung `sigma` wird eine quadratische Funktion in Abhängigkeit von der Anzahl an Steinen auf dem Spielfeld verwendet. Diese wird im folgenden \autoref{sec:pcsigma} hergeleitet.  Die Eingabe- und der Rückgabeparameter gleichen der Funktion `alphabeta`.

In [ ]:
debug_pc_count = 0

def probcut(state, depth, heuristic, alpha, beta):
    global debug_pc_count
    if state.game_over:
        return get_utility(state)
    if depth == 0:
        key = (state.board.tobytes(), state.turn, heuristic)
        if key in transposition_table:
            return transposition_table[key][0]
        debug_pc_count += 1
        h = heuristic(state)
        transposition_table[key] = (h, 0)
        return h

    if depth == PROBCUT_DEEP_DEPTH:
        num_p = state.num_pieces
        if num_p <= 58:
            if PROBCUT_DEEP_DEPTH == 4:
                sigma = -0.007429 + 0.00276 * num_p - 2.11821e-05 * num_p**2
            elif PROBCUT_DEEP_DEPTH == 3:
                sigma = -0.008665 + 0.00311 * num_p - 2.96811e-05 * num_p**2
            if beta < 1:
                bound = PERCENTILE * sigma + beta
                if probcut(state, PROBCUT_SHALLOW_DEPTH,
                           heuristic, -math.inf, bound) >= bound:
                    return beta
            if alpha > -1:
                bound = -PERCENTILE * sigma + alpha
                if probcut(state, PROBCUT_SHALLOW_DEPTH,
                           heuristic, bound, math.inf) <= bound:
                    return alpha

    moves = state.possible_moves
    child_states = [make_move(state, move) for move in moves]
    ordered_moves = []
    for child_state in child_states:
        key = (child_state.board.tobytes(), child_state.turn, heuristic)
        cached = transposition_table.get(key)
        if cached == None:
            debug_pc_count += 1
            cached = (heuristic(child_state), 0)
            transposition_table[key] = cached
        ordered_moves.append((cached[0], child_state, cached[1]))
    ordered_moves.sort(reverse=(state.turn == WHITE))

    if state.turn == WHITE:
        # maximizing
        utility = -math.inf
    else:
        # minimizing
        utility = math.inf

    for (_, tmp_state, cached_depth) in ordered_moves:
        tmp_utility = probcut(tmp_state, depth - 1, heuristic, alpha, beta)
        if depth - 1 > cached_depth:
            transposition_table[
                (tmp_state.board.tobytes(),
                 tmp_state.turn, heuristic)
            ] = (tmp_utility, depth -1)

        if state.turn == WHITE:
            # maximizing
            utility = max(utility, tmp_utility)
            alpha = max(alpha, utility)
        else:
            # minimizing
            utility = min(utility, tmp_utility)
            beta = min(beta, utility)
        if alpha >= beta:
            break  # alpha-beta pruning
    return utility

## Durchführen der Züge
Die folgenden Funktionen berechnen mithilfe einer angegebenen \ac{KI} Strategie den nächsten Zug und wenden diesen auf den übergebenen Zustand `state` an. Damit die Strategien nicht völlig deterministisch sind, und somit besser die Stärke der einzelnen Strategien und Heuristiken bestimmt werden kann, wird nicht immer der beste Zug ausgewählt, sondern stattdessen einer der Züge, die innerhalb eines festgelegten Abstands vom besten Zug liegen. Dieser Abstand wird als `SELECTION_TOLERANCE` definiert.

In [ ]:
SELECTION_TOLERANCE = 0.0001

### Suche mit fester Tiefe

Die Funktion `ai_make_move` ist die einfachste der Ausführungsfunktionen. Sie bewertet alle durch einen Zug vom Zustand `state` erreichbaren Spielpositionen und wählt aus diesen, wie oben beschrieben, einen der besten Züge aus. Die Bewertung der Spielzustände wird von der als Parameter übergebenen Funktion `ai` vorgenommen, welche eine der im vorherigen Abschnitt definierten Strategie-Funktionen sein kann. Für jeden Zustand wird die Strategie-Funktion genau einmal mit der Tiefe `depth-1` ausgeführt. Das `-1` wird hierbei verwendet, da
bereits in der Funktion `ai_make_move` selbst eine Iteration über die Kindzustände durchgeführt wird. Die Strategie-Funktion erhält außerdem den übergebenen Parameter `heuristic`, welcher eine der implementierten Heuristik-Funktionen sein kann.

In [ ]:
def ai_make_move(ai, state, depth, heuristic):
    global utilities
    if state.game_over:
        return
    scored_moves = []
    if state.turn == WHITE:
        # maximizing
        for move in state.possible_moves:
            new_state = make_move(state, move)
            utility = ai(new_state, depth-1, heuristic, -math.inf, math.inf)
            scored_moves.append((utility, move))
        best_score, _ = max(scored_moves)
    else:
        # minimizing
        for move in state.possible_moves:
            new_state = make_move(state, move)
            utility = ai(new_state, depth-1, heuristic, -math.inf, math.inf)
            scored_moves.append((utility, move))
        best_score, _ = min(scored_moves)
    utilities[state.turn] = best_score
    top_moves = [move for move in scored_moves
                 if abs(move[0] - best_score) <= SELECTION_TOLERANCE]
    best_move = random.choice(top_moves)[1]
    return make_move(state, best_move)

### Iterative Tiefensuche

Die Ausführungsfunktion `ai_make_move_id` unterscheidet sich von `ai_make_move` dadurch, dass eine iterative Tiefensuche durchgeführt wird. Dazu wird die Strategie-Funktion, statt nur einmal mit der vorgegebenen Tiefe aufgerufen zu werden, beginnend von 1 mit immer höherer Suchtiefe aufgerufen. Wird die Tiefe `depth` erreicht, so wird wie auch in `ai_make_move` einer der besten Züge ausgewählt. Durch die Verwendung eines Caches, der `transposition_table`, in den auf Alpha-Beta-Pruning basierenden Strategien, kann durch die Wiederverwendung der Ergebnisse vorheriger Aufrufe ein besseres Move Ordering vorgenommen werden, und somit die Effizienz des Alpha-Beta-Pruning gesteigert werden. Die Idee dabei ist, dass bei ausreichender Suchtiefe die durch besseres Move Ordering erzielten Ersparnisse den durch den mehrfachen Aufruf der Strategie-Funktionen verursachten, Mehraufwand übertreffen.

In [ ]:
def ai_make_move_id(ai, state, depth, heuristic):
    global utilities
    if state.game_over:
        return
    best_move = None
    cur_depth = 1
    while cur_depth <= depth:
        scored_moves = []
        if state.turn == WHITE:
            # maximizing
            for move in state.possible_moves:
                new_state = make_move(state, move)
                utility = ai(new_state, cur_depth-1,
                             heuristic, -math.inf, math.inf)
                scored_moves.append((utility, move))
            best_score, _ = max(scored_moves)
        else:
            # minimizing
            for move in state.possible_moves:
                new_state = make_move(state, move)
                utility = ai(new_state, cur_depth-1,
                             heuristic, -math.inf, math.inf)
                scored_moves.append((utility, move))
            best_score, _ = min(scored_moves)
        utilities[state.turn] = best_score
        top_moves = [move for move in scored_moves
                     if abs(move[0] - best_score) <= SELECTION_TOLERANCE]
        best_move = random.choice(top_moves)[1]
        cur_depth += 1
    return make_move(state, best_move)

### Zeitbeschränkte Tiefensuche

Je nach Spielsituation ist die Mobilität der Spieler unterschiedlich hoch. Dadurch unterscheidet sich auch die Anzahl der zu betrachtenden Spielzustände. Auch die Anzahl der durch Alpha-Beta-Pruning entfernten Zweige kann variieren. Bei konstanter Suchtiefe ist daher mit variablen Ausführungszeiten zu rechnen. Im Spiel gegen einen menschlichen Spieler ist es jedoch wünschenswert, eine maximale Zugdauer nicht zu überschreiten. Die verfügbare Zeit soll dabei dennoch effektiv für eine möglichst gute Entscheidung genutzt werden.

Das ist das Ziel der Ausführungsfunktion `ai_make_move_id_timelimited`, diese führt eine iterative Tiefensuche durch, kann jedoch nach jeder Iteration abbrechen und einen der bis dahin besten Züge wählen. Hierbei wird die Entscheidung zum Abbruch getroffen, wenn mit der nächsten Ausführung das durch den Parameter `timelimit` gegebene Zeitlimit voraussichtlich überschritten würde. Dafür wird die Dauer der nächsten Ausführung approximiert, indem bestimmt wird, um welchen Faktor sich die Ausführungszeit bei den letzten beiden Ausführungen geändert hat. Dieser Faktor `factor` wird dann mit der Dauer der letzten Ausführung multipliziert, um die Dauer der nächsten Ausführung zu schätzen. Zu beachten ist, dass die Funktion `ai_make_move_id_timelimited` nicht exakt die gleiche Schnittstelle hat, wie die anderen Ausführungsfunktionen. Der Parameter `depth` wurde hier durch das `timelimit` ersetzt. Dies ist beim Aufruf der Funktion zu beachten.

In [ ]:
def ai_make_move_id_timelimited(ai, state, timelimit, heuristic):
    global utilities
    if state.game_over:
        return
    best_move = None
    depth = 1
    last_time = 1
    second_last_time = 1
    factor = 0
    start = time.time()
    while (
        depth <= 64 - state.num_pieces and
        timelimit - (time.time() - start) >= factor * last_time
    ):
        last_time_start = time.time()
        scored_moves = []
        if state.turn == WHITE:
            # maximizing
            for move in state.possible_moves:
                new_state = make_move(state, move)
                utility = ai(new_state, depth-1,
                             heuristic, -math.inf, math.inf)
                scored_moves.append((utility, move))
            best_score, _ = max(scored_moves)
        else:
            # minimizing
            for move in state.possible_moves:
                new_state = make_move(state, move)
                utility = ai(new_state, depth-1,
                             heuristic, -math.inf, math.inf)
                scored_moves.append((utility, move))
            best_score, _ = min(scored_moves)
        utilities[state.turn] = best_score
        top_moves = [move for move in scored_moves
                     if abs(move[0] - best_score) <= timelimit]
        best_move = random.choice(top_moves)[1]
        second_last_time = last_time
        last_time = time.time() - last_time_start
        factor = min(last_time / second_last_time, 3)
        depth += 1
    print("Reached depth", depth-1, "in", time.time() - start, "seconds")
    return make_move(state, best_move)